In [ ]:
from willireply.data import enron
from willireply.features import features
from willireply.features.feature_extractor import FeatureExtractor

/usr/local/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, fbeta_score

from pathlib import Path
from tqdm.autonotebook import tqdm
from sklearn.linear_model import SGDRegressor
import numpy as np
import ipywidgets as widgets

# Standard Test

In [4]:
df_train = enron.get_dataframe('skilling-j', received_only=True)
np.log(1+features.words_in_body(df_train))

ValueError: ('skilling-j', 'is not in index folder')

In [19]:
from willireply.features import features

my_common_words = ['ASAP', 'please', 'could you']
subject_common_words_feature = lambda df: features.common_words_subject(df, ['?' '!'])
body_common_words_feature = lambda df: features.common_words_body(df, ['?', '!'])

fe = FeatureExtractor(
      subject_common_words_feature,
      body_common_words_feature,
      #features.thread_length,
      lambda df: np.log(1+features.words_in_body(df)),
      features.words_in_subject
                     )

In [5]:
#users = [f.stem for f in enron.ENRON_INDEX_FOLDER.iterdir() if enron.is_labeled(f.stem)]

In [20]:
users = ['skilling-j']

In [32]:
model = SGDRegressor(max_iter=100)

In [41]:
for user in tqdm(users[:20]):
    df_train = enron.get_dataframe(user, received_only=True)
    if len(df_train) == 0:
        print(user,"has no emails")
        continue
    X = fe.extract(df_train)
    y_true = fe.get_labels(df_train)
    if len(np.where(y_true)[0]) == 0:
        print(user,"has no replies")
        continue

    model.fit(X, y_true)

HBox(children=(IntProgress(value=0, max=1), HTML(value='')))

In [37]:
#df_train = enron.get_dataframe('may-l', received_only=True)
df_validate = enron.get_dataframe('skilling-j', received_only=True)

In [42]:
y_pred = model.predict(fe.extract(df_validate)) > 0# Not sure best place to set this predict...
y_true = fe.get_labels(df_validate)

print(classification_report(y_true, y_pred, target_names=["no reply", "reply"]))
print('f_2 = %s' % fbeta_score(y_true, y_pred, 2, labels=['no reply', 'reply'], pos_label=1))

             precision    recall  f1-score   support

   no reply       0.97      0.34      0.50      3411
      reply       0.03      0.66      0.06       109

avg / total       0.94      0.35      0.49      3520

f_2 = 0.1305293691080493


In [27]:
df_all = enron.get_dataframe(users[0])

In [28]:
df_all.loc[61].body.split('-----Original Message-----')[0]

'Confirmed for 10:30am on Wed Dec 13.  Your place our ours?\n\nThanks Sherri.\n\n'

In [29]:
df2 = df_all.copy()
df2['body'] = df_all['body'].apply(lambda x: x.split('-----Original Message-----')[0])
print(df2.loc[61].body)

Confirmed for 10:30am on Wed Dec 13.  Your place our ours?

Thanks Sherri.




In [1]:
missed = df_validate.iloc[np.where(y_true > y_pred)[0]]

labels = []
left = []
right = []

for idx, row in missed.iterrows():
    left.append(row.body)
    labels.append(f'({idx}) {row.subject}')
    right.append(df_all.loc[row.reply_id].body)
    
accordion = widgets.Accordion(children=[
    widgets.Box( [widgets.Textarea(l, layout={'height': '500px'}), widgets.Textarea(r, layout={'height': '500px'})] )
    for (l, r) in zip(left, right)
])
for i, l in enumerate(labels):
    accordion.set_title(i, l)
accordion

NameError: name 'df_validate' is not defined